In [1]:
import numpy as np
import os

# File inputs
path='data/'
f=sorted([f for f in os.listdir(path) if f.startswith('eof_ts')],key=str.lower)
x=[None]*len(f)
for i in np.arange(0,len(f)):
    print(f[i][7:])
    x[i]=np.loadtxt(path+f[i])

msl-01
msl-02
msl-03
msl-04
msl-06
msl-07
si10-00
si10-01
si10-04
si10-05
si10-06
si10-07
sst-00
sst-03
sst-04
sst-05
sst-06
sst-07
t2m-02
t2m-03
t2m-04
t2m-06
t2m-07
t2m-08


In [2]:
# Hyperparameters for phase space reconstruction and CCM
rlag=1
rdim=3
maxdelay=3

# Parameters for phase space reconstruction and CCM
length=x[0].shape[0]
reconst_length=length-rlag*(rdim-1)
reconst_range=np.arange(rlag*(rdim-1),length)
delay=np.arange(-maxdelay,maxdelay+1)

# The library for cross mapping is a little shorter
lib=np.arange(max(rlag*(rdim-1),maxdelay),length-maxdelay)

print('length='+str(length))
print('reconstucted length='+str(reconst_length))
print('reconstucted range: '+str(reconst_range[0])+' ~ '+str(reconst_range[-1]))
print('library: '+str(lib[0])+' ~ '+str(lib[-1]))
print('CCM delay: '+str(delay[0])+' ~ '+str(delay[-1]))

length=480
reconstucted length=478
reconstucted range: 2 ~ 479
library: 3 ~ 476
CCM delay: -3 ~ 3


In [4]:
import sys
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import csv

# Write outputs to a .csv file
with open('ccm_python.csv',mode='w') as output_file:
    ccm_writer=csv.writer(output_file,delimiter=',')
    ccm_writer.writerow(['Effect','Cause','Timelag','Strength','Method'])

    # CCM computation
    for j in np.arange(0,len(f)):
    
        # Phase space reconstruction for the effect variable x[j]
        reconst=np.zeros([reconst_length,rdim])
        for d in np.arange(0,rdim):
            reconst[:,d]=x[j][d*rlag:d*rlag+reconst_length]

        # Compute Euclidean distances in the reconstructed phase space
        dist=cdist(reconst[lib-rlag*(rdim-1),:],reconst[lib-rlag*(rdim-1),:])
    
        # Exclude the pair of identical phase points
        for k in np.arange(0,len(lib)):
            dist[k,k]=sys.maxsize
                
        # Find the rdim+1 nearest neighbors and assign weights
        index=np.argsort(dist,axis=0)
        sort_dist=np.sort(dist,axis=0)
        weight=np.exp(-sort_dist[:rdim+1]/sort_dist[0])
    
        for i in np.arange(0,len(f)):
        
            # The first dimension is {correlation coefficient, p-value};
            # the second dimension is delay.
            cmskill=np.zeros([2,len(delay)])
            for l in np.arange(0,len(delay)):
                estimate=np.average(x[i][lib[index[:rdim+1]]-delay[l]],axis=0,weights=weight)
                cmskill[:,l]=pearsonr(estimate,x[i][lib-delay[l]])
                
            # Find the optimal delay and corresponding CCM skill
            idx=np.argsort(cmskill[1,:])
            optdelay=delay[idx[0]]
            p=cmskill[1,idx[0]]
            
            # Only preserve meaningful and significant results
            if optdelay>0 and p<0.05:
                ccm_writer.writerow([f[j][7:],f[i][7:],str(optdelay),str(p),'CCM'])